<a href="https://colab.research.google.com/github/DominikBurkert/PyTorch/blob/master/PredictiveRework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Umgebungsvariablen laden

In [ ]:
from google.colab import output
#I don't know why nbextensions don't work with python3.6 
with output.temporary():
  !pip install --upgrade git+https://github.com/Kreijstal/colab_inspector.git 
  !python2.7 -m pip install --upgrade git+https://github.com/blois/colab_inspector.git #yes, really.
  !jupyter nbextension install --py inspector

import inspector

# open a scratch cell (Ctrl+Alt+N)
# run there
# inspector.watch_globals()

# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Daten laden

In [ ]:
from google.colab import files #daten werden nur für die aktuelle sizung geladen!
uploaded = files.upload()

Saving cause_small.csv to cause_small.csv


In [ ]:
"""Inputdaten für das Netz"""
# Das sind die Beispiele aus der Transformer Doku
# https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

#input = torch.rand((10, 32, 512))
#target = torch.rand((20, 32, 512))

"""Input-Daten laden"""
#input data = Symptome
input = pd.read_csv('symptom_small.csv')
input = input.iloc[:, :-1] #letze spalte löschen (da stehen irgendwelche Buchstaben und Zahlen drin. )
"""
ist die letzte spalte wichtig? ERnst fragen.
"""
#input   error_bad_lines=False
#input ist ein dataframe -> input in tensor umwandeln
input = torch.tensor(input.values)
#transformer benötigt eine 3dimensionale matrix als input
print(input.size())
input = input.unsqueeze(0)
print(input.size())

torch.Size([496, 5182])
torch.Size([1, 496, 5182])


In [ ]:
""" Target Daten laden  """
target = pd.read_csv('cause_small.csv')
target = target.iloc[:, :-1] #letze spalte löschen (da stehen irgendwelche Buchstaben und Zahlen drin. )
print(target.shape)
"""
ist die letzte spalte wichtig? ERnst fragen.
"""
#target und input muss diesselbe anzahl an features haben wie embedding_size.
auffüllen = np.zeros((496, 2891), dtype=int)
auffüllen = pd.DataFrame(data=auffüllen)
auffüllen
print(auffüllen.shape)
target = pd.concat([target, auffüllen], axis=1)
target
target.shape
#target ist ein dataframe -> input in tensor umwandeln
target = torch.tensor(target.values)
#transformer benötigt eine 3dimensionale matrix als input
print(target.size())
target = target.unsqueeze(0)
print(target.size())

(496, 2291)
(496, 2891)
torch.Size([496, 5182])
torch.Size([1, 496, 5182])


In [ ]:
print("input.shape: ", input.shape)
print("target.shape: ", target.shape)
print("Input: \n", input)
print("Target: \n", target)

input.shape:  torch.Size([1, 496, 5182])
target.shape:  torch.Size([1, 496, 5182])
Input: 
 tensor([[[0, 0, 1,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]])
Target: 
 tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]])


In [ ]:
#transformer benötigt die Daten als Float 
target = target.float()
input = input.float()
print(target)

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


# Model erstellen

In [ ]:
model = torch.nn.Transformer(d_model=5182, nhead=2, num_encoder_layers=6, 
                     num_decoder_layers=3,  dropout=0.1, 
                     activation='relu', custom_encoder=None, custom_decoder=None)

#wie soll der fehler berechnet werden? (softmax, mean-squared-error(MSELoss), CrossEntropyLoss)
criterion = nn.MSELoss()

print(model)

Transformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=5182, out_features=5182, bias=True)
        )
        (linear1): Linear(in_features=5182, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=5182, bias=True)
        (norm1): LayerNorm((5182,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((5182,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=5182, out_features=5182, bias=True)
        )
        (linear1): Linear(in_features=5182, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (l

In [ ]:
print(input.shape)
print(target.shape)

torch.Size([1, 496, 5182])
torch.Size([1, 496, 5182])


In [ ]:
for i in range(100):  #100 epochen trainieren
  out = model(input, target) #einmal das netz durchlaufen
  loss = criterion(out, target) #vergleich von output (vorhersage des netzes) und target(echte Werte)-> Berechnung des Fehlers
  print(loss)

  model.zero_grad()
  loss.backward()
  optimizer = optim.SGD(model.parameters(), lr=0.10)
  optimizer.step()

tensor(0.9965, grad_fn=<MseLossBackward>)


# Überprüfung

In [ ]:
print("Vorhersage: \n", torch.round(out[0,5]))
print("Ziel: \n", target[0,5])

NameError: ignored